### This Notebook Shows how I Generate the Basic Strategy Charts from chart_generation.py and the Exact Expected Values from the Simulation

Provides a good visual of how I read the data from the .csv's generated from the simulation and wrangle them into dataframes

In [1]:
# import statements; the %autoreload stuff is just to make sure the modules load in correctly as they can be finicky
%load_ext autoreload
%autoreload 2

from pathlib import Path

from blackjack.helper.io import taking_generated_chart_path
from blackjack.helper.io import DEFAULT_CHART_OUTPUT_PATH
from blackjack.helper.io import DEFAULT_DATA_OUTPUT_PATH
from blackjack.helper.io import DEFAULT_DATA_SPLIT_OUTPUT_PATH

import pandas as pd


\

Creating the master dataframe that shows all the expected values for...\
&nbsp;&nbsp;&nbsp;&nbsp;every choice ['hit','stand','double','surrender']\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;for...\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;every combination of 'player hand total' and 'dealer face up card'

In [2]:
dataframe = pd.read_csv(DEFAULT_DATA_OUTPUT_PATH)
dataframe.head()

,player hand total,player hand texture,dealer face up,player choice,expected value
0,20,hard,11,stand,2.67
1,20,hard,11,hit,-22.53
2,20,hard,11,double,-37.31
3,20,hard,11,surrender,-16.36
4,19,hard,11,stand,-4.62


In [3]:
# This organizes the data into a pivot table with an hierarchical index, easier to read

# uncomment the row below to see the entire table
# pd.set_option('display.max_rows', 2000)
master_dataframe = pd.pivot_table(dataframe, values='expected value', index=['player hand texture', 'player hand total', 'dealer face up'], columns=['player choice'])
master_dataframe = master_dataframe.reindex(columns=['hit','double','stand','surrender'])
master_dataframe

player choice                                          hit  double  stand  \
player hand texture player hand total dealer face up                        
hard                4                 2              -2.81  -14.50  -7.41   
                                      3              -1.97  -12.15  -6.17   
                                      4              -1.27  -10.24  -5.13   
                                      5              -0.17   -8.19  -3.67   
                                      6               0.51   -5.91  -2.93   
...                                                    ...     ...    ...   
soft                20                7               6.28    9.30  19.29   
                                      8               4.81    6.93  19.74   
                                      9               2.87    3.61  19.01   
                                      10             -1.38   -2.39  10.90   
                                      11             -7.19   -8.62   2.72   

player choice                                         surrender  
player hand texture player hand total dealer face up             
hard                4                 2                  -12.50  
                                      3                  -12.50  
                                      4                  -12.50  
                                      5                  -12.50  
                                      6                  -12.50  
...                                                         ...  
soft                20                7                  -12.50  
                                      8                  -12.50  
                                      9                  -12.50  
                                      10                 -13.47  
                                      11                 -16.32  

[260 rows x 4 columns]

\

Creating the decision matrix for HARD player hand totals\
I.e. the optimal decision to make for every HARD player total

In [4]:
hard_dataframe = pd.read_csv(DEFAULT_DATA_OUTPUT_PATH)
hard_dataframe = hard_dataframe[hard_dataframe["player hand texture"] == "hard"]

# this returns the line returns the max expected value for each 'player hand total' and 'dealer face up' combination (from here on referred to as 'case')
hard_dataframe_1 = hard_dataframe.groupby(by=["player hand total","dealer face up"], as_index=False).max("average expected value")
hard_dataframe_1

,player hand total,dealer face up,expected value
0,4,2,-2.81
1,4,3,-1.97
2,4,4,-1.27
3,4,5,-0.17
4,4,6,0.51
...,...,...,...
165,20,7,19.30
166,20,8,19.83
167,20,9,18.97
168,20,10,10.94


In [5]:
# by merging hard_dataframe_1 (the dataset with the max expected value for each case) with the original hard_dataframe (that has all the rows)
# we get a dataframe that has all the original columns, but only the max 'player choice' for each case

# NOTE: In the **rare** instances that there are ties for the max expected values, .drop_duplicates line will delete the second instance...
hard_dataframe_2 = pd.merge(left=hard_dataframe,right=hard_dataframe_1,how='inner').drop_duplicates(["player hand total","dealer face up"])
hard_dataframe_2

,player hand total,player hand texture,dealer face up,player choice,expected value
0,20,hard,11,stand,2.67
1,19,hard,11,stand,-4.62
2,18,hard,11,stand,-11.57
3,17,hard,11,surrender,-16.36
4,16,hard,11,surrender,-16.35
...,...,...,...,...,...
166,8,hard,2,hit,-0.43
167,7,hard,2,hit,-2.75
168,6,hard,2,hit,-3.43
169,5,hard,2,hit,-2.99


In [6]:
# this cell is reorganizing hard_dataframe_2 (the dataframe we got by merging) to be human-readable-friendly
hard_decision_matrix = hard_dataframe_2.pivot(index='player hand total',columns='dealer face up',values='player choice')
hard_decision_matrix.index.names = ['Player Hard Total']
hard_decision_matrix.columns.names = ['Dealer Face Up']
hard_decision_matrix.rename(columns={11: 'A'}, inplace=True)
hard_decision_matrix

Dealer Face Up,2,3,4,5,6,7,8,9,10,A
Player Hard Total,,,,,,,,,,
4,hit,hit,hit,hit,hit,hit,hit,hit,hit,hit
5,hit,hit,hit,hit,hit,hit,hit,hit,hit,hit
6,hit,hit,hit,hit,hit,hit,hit,hit,hit,hit
7,hit,hit,hit,hit,hit,hit,hit,hit,hit,hit
8,hit,hit,hit,hit,hit,hit,hit,hit,hit,hit
9,double,double,double,double,double,hit,hit,hit,hit,hit
10,double,double,double,double,double,double,double,double,hit,hit
11,double,double,double,double,double,double,double,double,double,double
12,hit,hit,stand,stand,stand,hit,hit,hit,hit,hit


\

Creating the decision matrix for SOFT player hand totals\
I.e. the optimal decision to make for every SOFT player total

In [7]:
# follow the exact same steps as the hard_dataframe above, except we are extracting the "soft" totals this time
soft_dataframe = pd.read_csv(DEFAULT_DATA_OUTPUT_PATH)
soft_dataframe = soft_dataframe[soft_dataframe["player hand texture"] == "soft"]
soft_dataframe_1 = soft_dataframe.groupby(by=["player hand total","dealer face up"], as_index=False).max("average expected value")
soft_dataframe_2 = pd.merge(left=soft_dataframe,right=soft_dataframe_1,how='inner').drop_duplicates(["player hand total","dealer face up"])
soft_decision_matrix = soft_dataframe_2.pivot(index='player hand total',columns='dealer face up',values='player choice')
soft_decision_matrix

dealer face up,2,3,4,5,6,7,8,9,10,11
player hand total,,,,,,,,,,
12,hit,hit,hit,hit,double,hit,hit,hit,hit,hit
13,hit,hit,hit,double,double,hit,hit,hit,hit,hit
14,hit,hit,hit,double,double,hit,hit,hit,hit,hit
15,hit,hit,hit,double,double,hit,hit,hit,hit,hit
16,hit,hit,double,double,double,hit,hit,hit,hit,hit
17,hit,double,double,double,double,hit,hit,hit,hit,hit
18,stand,double,double,double,double,stand,stand,hit,hit,hit
19,stand,stand,stand,stand,double,stand,stand,stand,stand,stand
20,stand,stand,stand,stand,stand,stand,stand,stand,stand,stand


In [8]:
# this cell is reorganizing soft_decision_matrix to be human-readable-friendly
soft_decision_matrix.rename(index={12: 'A,A', 13: 'A,2', 14: 'A,3', 15: 'A,4', 16: 'A,5', 17: 'A,6', 18: 'A,7', 19: 'A,8', 20: 'A,9'}, inplace= True)
soft_decision_matrix.index.names = ['Player Hand']
soft_decision_matrix.columns.names = ['Dealer Face Up']
soft_decision_matrix.rename(columns={11: 'A'}, inplace=True)
soft_decision_matrix

Dealer Face Up,2,3,4,5,6,7,8,9,10,A
Player Hand,,,,,,,,,,
"A,A",hit,hit,hit,hit,double,hit,hit,hit,hit,hit
"A,2",hit,hit,hit,double,double,hit,hit,hit,hit,hit
"A,3",hit,hit,hit,double,double,hit,hit,hit,hit,hit
"A,4",hit,hit,hit,double,double,hit,hit,hit,hit,hit
"A,5",hit,hit,double,double,double,hit,hit,hit,hit,hit
"A,6",hit,double,double,double,double,hit,hit,hit,hit,hit
"A,7",stand,double,double,double,double,stand,stand,hit,hit,hit
"A,8",stand,stand,stand,stand,double,stand,stand,stand,stand,stand
"A,9",stand,stand,stand,stand,stand,stand,stand,stand,stand,stand


\

Creating the decision matrix for SPLIT player hand totals\
I.e. the optimal decision to make for every SPLIT player total

In [9]:
# when loading in the split data set, NOTE: this is a different csv than what was used for hard and soft hands, we see that it is organized a bit differently
split = pd.read_csv(DEFAULT_DATA_SPLIT_OUTPUT_PATH)
split.head()

,player hand total,player hand texture,dealer face up,player choice,expected value
0,20,split,11,split,-21.54
1,18,split,11,split,-12.18
2,16,split,11,split,-16.85
3,14,split,11,split,-20.44
4,12,split,11,split,-20.16


In [10]:
# this cell is unncessary, but it makes the split dataframe more human-readable

# pd.set_option('display.max_rows', 2000)
split_pivot = pd.pivot_table(split, values='expected value', index=['player hand texture', 'player hand total', 'dealer face up'])
split_pivot

expected value
player hand texture player hand total dealer face up                
split               2                 2                        12.39
                                      3                        13.15
                                      4                        14.51
                                      5                        15.85
                                      6                        17.01
...                                                              ...
                    20                7                         3.03
                                      8                        -5.45
                                      9                       -14.94
                                      10                      -18.80
                                      11                      -21.54

[100 rows x 1 columns]

To answer the question if we should split or not, we have to compare the expected value we get from the split dataframe and the max associated hard dataframe\
This is because every splittable hand: {20: 10+10, 18: 9+9, 16: 8+8,... etc} is a hard value hand...\
&nbsp;&nbsp;&nbsp;&nbsp;with the exception of A,A which is technically a 'soft 12', but that edge case is handled later.\
\
For example, if the player has a 7+7, we are going to check the expected value for splitting a 7+7\
&nbsp;&nbsp;&nbsp;&nbsp;let's say that splitting a 7+7 has an expected value of 5.00\
Now we will check if the 'hard 14' (which a 7+7 classifies as) optimal / max decision has a higher value than 5.00\
&nbsp;&nbsp;&nbsp;&nbsp;let's say that the optimal decision for a 'hard 14' is 'stand', and the associated expected value of 'stand' is 4.70\
\
In this example, we would say that you should split a 7+7 rather than playing it as a 'hard 14'

In [11]:
# showing the hard dataframe that contains the optimal / max 'player choice' and associated 'expected value'
hard_dataframe_2

,player hand total,player hand texture,dealer face up,player choice,expected value
0,20,hard,11,stand,2.67
1,19,hard,11,stand,-4.62
2,18,hard,11,stand,-11.57
3,17,hard,11,surrender,-16.36
4,16,hard,11,surrender,-16.35
...,...,...,...,...,...
166,8,hard,2,hit,-0.43
167,7,hard,2,hit,-2.75
168,6,hard,2,hit,-3.43
169,5,hard,2,hit,-2.99


In [12]:
# using loc we can return the expected value for when we search the hard_dataframe_2 for a certain case: NOTE the return float is technically a series object...
# we'll need to convert it to a float later
hard_dataframe_2.loc[(hard_dataframe_2['player hand total'] == 20) & (hard_dataframe_2['dealer face up'] == 11), "expected value"]

0    2.67
Name: expected value, dtype: float64

In [13]:
# using loc we can return the expected value for when we search the split for a certain case: NOTE the return float is technically a series object...
# we'll need to convert it to a float later
split.loc[(split['player hand total'] == 20) & (split['dealer face up'] == 11), "expected value"]

0   -21.54
Name: expected value, dtype: float64

In [14]:
# these functions perform the case search in their specified dataframe (hard_dataframe_2, split, or soft_dataframe_2) and return the expected value (as a series)
# technically you could just do the search directly in the for loops 2 cells down, but I think using these functions makes it easier to understand what is happening

def hard_search(player_hand_total, dealer_face_up):
    search = hard_dataframe_2.loc[(hard_dataframe_2['player hand total'] == player_hand_total) & (hard_dataframe_2['dealer face up'] == dealer_face_up), "expected value"]
    return search

def split_search(player_hand_total, dealer_face_up):
    search = split.loc[(split['player hand total'] == player_hand_total) & (split['dealer face up'] == dealer_face_up), "expected value"]
    return search

def soft_search(player_hand_total, dealer_face_up):
    search = soft_dataframe_2.loc[(soft_dataframe_2['player hand total'] == player_hand_total) & (soft_dataframe_2['dealer face up'] == dealer_face_up), "expected value"]
    return search


In [15]:
# just testing that it is returning what we want
# because the search is technically returning a series of length one, we use .iloc[0] to directly extract that value
float(hard_search(20,11).iloc[0])

2.67

In [16]:
# these are all the cases we will need to compare
split_numbers = [20,18,16,14,12,10,8,6,4]
dealer_numbers = [11,10,9,8,7,6,5,4,3,2]

# we will be storing the results of the search & compare as List[Tuple]; with the tuple being ('player hand total','dealer face up', 'should you split?')
split_yes_no = []

# comparing the max values from the hard_dataframe_2 and the split dataframe. Storing the values as a tuple and append to a list
for x in dealer_numbers:
    for y in split_numbers:
        if float(split_search(y,x).iloc[0]) > float(hard_search(y,x).iloc[0]):
            split_yes_no.append((y,x,'yes'))
        else:
            split_yes_no.append((y,x,'no'))

# same comparison as above but checking the 'soft 12' case from the soft_dataframe_2; 'soft 12' is A,A and the A,A case is in the soft database not the hard database
for z in dealer_numbers:
    # in split, case A,A is represented by a 'player hand' of 2;;; In soft, case A,A is represented by a 'player hand' of 12
    if float(split_search(player_hand_total=2,dealer_face_up=z).iloc[0]) > float(soft_search(player_hand_total=12,dealer_face_up=z).iloc[0]):
        split_yes_no.append((2,z,'yes'))
    else:
        split_yes_no.append((2,z,'no'))

# just ensuring that the length the list is 100 (it should be 100)
len(split_yes_no)

100

In [17]:
# converting the List[Tuple] into a dataframe
split_yes_no_df = pd.DataFrame(split_yes_no, columns=["player hand total","dealer face up","split?"])
split_yes_no_df

,player hand total,dealer face up,split?
0,20,11,no
1,18,11,no
2,16,11,no
3,14,11,no
4,12,11,no
...,...,...,...
95,2,6,yes
96,2,5,yes
97,2,4,yes
98,2,3,yes


In [18]:
# making the dataframe human-readable and in the same form as the previous decision matrices
split_decision_matrix = pd.pivot(split_yes_no_df, index=['player hand total'], columns=['dealer face up'], values=['split?'])

split_decision_matrix.rename(index={2: 'A,A', 4: '2,2', 6: '3,3', 8: '4,4', 10: '5,5', 12: '6,6', 14: '7,7', 16: '8,8', 18: '9,9', 20: '10,10'}, inplace= True)
split_decision_matrix.index.names = ['Player Hand']
split_decision_matrix.rename(columns={11: 'A'}, inplace=True)

# for some reason this removes the ugly 'split?' header caption; I suppose it is technically slicing, but whatever
split_decision_matrix = split_decision_matrix['split?']
split_decision_matrix.columns.names = ['Dealer Face Up']

split_decision_matrix

Dealer Face Up,2,3,4,5,6,7,8,9,10,A
Player Hand,,,,,,,,,,
"A,A",yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
"2,2",yes,yes,yes,yes,yes,yes,yes,no,no,no
"3,3",yes,yes,yes,yes,yes,yes,no,no,no,no
"4,4",no,no,no,yes,yes,no,no,no,no,no
"5,5",no,no,no,no,no,no,no,no,no,no
"6,6",yes,yes,yes,yes,yes,no,no,no,no,no
"7,7",yes,yes,yes,yes,yes,yes,no,no,no,no
"8,8",yes,yes,yes,yes,yes,yes,yes,yes,yes,no
"9,9",yes,yes,yes,yes,yes,no,yes,yes,no,no


\

Styling and Exporting DataFrames to an HTML file

In [19]:
# this function colors all the player options 'hit','stand','double','surrender'
def color_choice(value):
    color = None
    if value == 'hit':
        color = '#9896f1'
    elif value == 'stand':
        color = '#d59bf6'
    elif value == 'double':
        color = '#edb1f1'
    elif value == 'surrender':
        color = '#6643b5'
    
    return f'background-color: {color}'


In [20]:
hard_styled = hard_decision_matrix.style.map(color_choice).set_table_attributes('style="border-collapse:collapse"').set_table_styles([
                                                          {"selector": "th",
                                                           "props": [("background-color","#d5eeff"), ("text-align","left"), ('color','black'), ('border','none')]},

                                                          {"selector": "td",
                                                           "props": [("text-align","left"), ('color','black'), ('border','none'), ('width', '75px')]}
])

hard_styled


Dealer Face Up,2,3,4,5,6,7,8,9,10,A
Player Hard Total,,,,,,,,,,
4,hit,hit,hit,hit,hit,hit,hit,hit,hit,hit
5,hit,hit,hit,hit,hit,hit,hit,hit,hit,hit
6,hit,hit,hit,hit,hit,hit,hit,hit,hit,hit
7,hit,hit,hit,hit,hit,hit,hit,hit,hit,hit
8,hit,hit,hit,hit,hit,hit,hit,hit,hit,hit
9,double,double,double,double,double,hit,hit,hit,hit,hit
10,double,double,double,double,double,double,double,double,hit,hit
11,double,double,double,double,double,double,double,double,double,double
12,hit,hit,stand,stand,stand,hit,hit,hit,hit,hit


In [21]:
soft_styled = soft_decision_matrix.style.map(color_choice).set_table_attributes('style="border-collapse:collapse"').set_table_styles([
                                                          {"selector": "th",
                                                           "props": [("background-color","#d5eeff"), ("text-align","left"), ('color','black'), ('border','none')]},

                                                          {"selector": "td",
                                                           "props": [("text-align","left"), ('color','black'), ('border','none'), ('width', '75px')]}
])

soft_styled

Dealer Face Up,2,3,4,5,6,7,8,9,10,A
Player Hand,,,,,,,,,,
"A,A",hit,hit,hit,hit,double,hit,hit,hit,hit,hit
"A,2",hit,hit,hit,double,double,hit,hit,hit,hit,hit
"A,3",hit,hit,hit,double,double,hit,hit,hit,hit,hit
"A,4",hit,hit,hit,double,double,hit,hit,hit,hit,hit
"A,5",hit,hit,double,double,double,hit,hit,hit,hit,hit
"A,6",hit,double,double,double,double,hit,hit,hit,hit,hit
"A,7",stand,double,double,double,double,stand,stand,hit,hit,hit
"A,8",stand,stand,stand,stand,double,stand,stand,stand,stand,stand
"A,9",stand,stand,stand,stand,stand,stand,stand,stand,stand,stand


In [22]:
# this function colors all the player options 'hit','stand','double','surrender'
def split_color_choice(value):
    color = None
    if value == 'no':
        color = '#d59bf6'
    elif value == 'yes':
        color = '#edb1f1'
    
    return f'background-color: {color}'

In [23]:
split_styled = split_decision_matrix.style.map(split_color_choice).set_table_attributes('style="border-collapse:collapse"').set_table_styles([
                                                          {"selector": "th",
                                                           "props": [("background-color","#d5eeff"), ("text-align","left"), ('color','black'), ('border','none')]},

                                                          {"selector": "td",
                                                           "props": [("text-align","left"), ('color','black'), ('border','none'), ('width', '75px')]}
])

split_styled

Dealer Face Up,2,3,4,5,6,7,8,9,10,A
Player Hand,,,,,,,,,,
"A,A",yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
"2,2",yes,yes,yes,yes,yes,yes,yes,no,no,no
"3,3",yes,yes,yes,yes,yes,yes,no,no,no,no
"4,4",no,no,no,yes,yes,no,no,no,no,no
"5,5",no,no,no,no,no,no,no,no,no,no
"6,6",yes,yes,yes,yes,yes,no,no,no,no,no
"7,7",yes,yes,yes,yes,yes,yes,no,no,no,no
"8,8",yes,yes,yes,yes,yes,yes,yes,yes,yes,no
"9,9",yes,yes,yes,yes,yes,no,yes,yes,no,no


Uncomment the cell below to create an HTML basic strategy chart from this notebook; though main.py already creates one after a running a simulation

In [24]:
#taking_generated_chart_path(file_path=DEFAULT_CHART_OUTPUT_PATH, hard_styled=hard_styled, soft_styled=soft_styled, split_styled=split_styled, rules_styled=rules_styled)
